<a href="https://colab.research.google.com/github/cw118/domain-adapted-nmt/blob/main/2_fr_domain_adapted_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Domain-adapted NMT

## Training NMT models

In [1]:
!pip3 install OpenNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 24.2 MB/s eta 0:00:00


In [2]:
# change into folder where prepared datasets were saved in the text processing step
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/domain-adapted-nmt/nmt-tools

/content/drive/MyDrive/domain-adapted-nmt/nmt-tools


### Train the general/base model

In [4]:
# corpora generated from step 1: https://drive.google.com/drive/folders/1fVe2e2MvT2CCTpSSDrBkykoR-7JKy-w4?usp=sharing
config = '''# config.yaml


## where the samples will be written
save_data: run

# train the general/base model first
data:
    corpus_1:
        path_src: corpora/enfr/en-fr-general.en-filtered.en.subword.train
        path_tgt: corpora/enfr/en-fr-general.fr-filtered.fr.subword.train
        transforms: [filtertoolong]
        weight: 1
    valid:
        path_src: corpora/enfr/en-fr-general.en-filtered.en.subword.dev
        path_tgt: corpora/enfr/en-fr-general.fr-filtered.fr.subword.dev
        transforms: [filtertoolong]

# vocab files generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# vocabulary size: should be same as in sentencepiece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source-general.model
tgt_subword_model: target-general.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model-base.enfr

# Stop training if it does not improve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 2000

# To save space, limit checkpoints to last n
# keep_checkpoint: 4

seed: 3435

# For fine-tuning, add up the required steps to the original steps
train_steps: 10000

# Default: 10000 - Run validation after n steps
valid_steps: 2000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 2000
report_every: 200

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [5]:
!nproc --all

2


In [6]:
# match -num_threads to number of CPUs to increase speed
# -1 for -n_sample to use entire corpus when building vocab
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

2024-01-24 00:20:11.220270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 00:20:11.220332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 00:20:11.222161: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 00:20:11.232645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 00:20:12.871031: W tensorflow/compiler/tf2

In [7]:
# once runtime type is changed to GPU, check that the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-495950c7-d9bc-9d3d-dd36-adffe3439b91)


In [8]:
# check that the GPU is visible to PyTorch
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0] / 1024**2, "out of", gpu_memory[1] / 1024**2)

True
Tesla T4
Free GPU memory: 14999.0625 out of 15102.0625


In [9]:
# clear the models directory for a fresh start
!rm -rf /content/drive/MyDrive/nmt-tools/models

In [10]:
# train NMT model
!onmt_train -config config.yaml

2024-01-24 00:20:52.369413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 00:20:52.369470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 00:20:52.371983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 00:20:52.382420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 00:20:53.947009: W tensorflow/compiler/tf2

In [35]:
# in case Colab is suddenly unable to navigate through directories
import os
path = '/content/drive/MyDrive/domain-adapted-nmt/nmt-tools'
os.chdir(path)

In [24]:
# -gpu 0 to use gpu
!onmt_translate -model models/model-base.enfr_step_10000.pt -src corpora/enfr/en-fr-general.en-filtered.en.subword.test -output general-fr.translated -gpu 0 -min_length 1

2024-01-24 02:44:04.733112: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 02:44:04.733175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 02:44:04.735231: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 02:44:04.767921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 02:44:06.891483: W tensorflow/compiler/tf2

In [25]:
!head -n 5 general-fr.translated

▁Il ▁ne ▁faut ▁pas ▁oublier ▁que ▁ 1 ▁ 2 6 0 ▁prisonniers ▁marocains ▁sont ▁toujours ▁détenus ▁dans ▁des ▁prisons ▁du ▁Front ▁POLISARIO , ▁où ▁ils ▁ont ▁été ▁maintenus ▁pendant ▁plus ▁de ▁ 2 5 ▁ans ▁en ▁violation ▁flagrante ▁du ▁droit ▁international ▁humanitaire .
▁Fournir ▁au ▁Comité ▁des ▁exemplaires ▁du ▁texte ▁de ▁la ▁Convention ▁relative ▁aux ▁droits ▁de ▁l ' enfant ▁dans ▁toutes ▁les ▁langues ▁officielles ▁ou ▁dans ▁d ' autres ▁langues ▁ou ▁dialectes , ▁lorsque ▁cela ▁est ▁disponible .
▁Nous ▁réaffirmons ▁que ▁les ▁lois ▁en ▁vigueur ▁dans ▁le ▁Sultanat ▁garantissent ▁la ▁protection ▁des ▁droits ▁de ▁l ' homme , ▁y ▁compris ▁les ▁droits ▁de ▁l ' enfant , ▁en ▁particulier ▁en ▁ce ▁qui ▁concerne ▁l ' implication ▁d ' enfants ▁dans ▁les ▁conflits ▁armés .
▁Conformément ▁à ▁l ' accord ▁auquel ▁le ▁Conseil ▁est ▁parvenu ▁lors ▁de ▁ses ▁consultations ▁préalables , ▁j ' invite ▁les ▁membres ▁du ▁Conseil ▁à ▁poursuivre ▁l ' examen ▁de ▁la ▁question .
▁Décisions ▁et ▁recommandations


In [15]:
!pip3 install --upgrade -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00


In [36]:
# desubword translated file
!python3 MT-Preparation/subwording/3-desubword.py target-general.model general-fr.translated

Done desubwording! Output: general-fr.translated.desubword


In [37]:
!head -n 5 general-fr.translated.desubword

Il ne faut pas oublier que 1 260 prisonniers marocains sont toujours détenus dans des prisons du Front POLISARIO, où ils ont été maintenus pendant plus de 25 ans en violation flagrante du droit international humanitaire.
Fournir au Comité des exemplaires du texte de la Convention relative aux droits de l'enfant dans toutes les langues officielles ou dans d'autres langues ou dialectes, lorsque cela est disponible.
Nous réaffirmons que les lois en vigueur dans le Sultanat garantissent la protection des droits de l'homme, y compris les droits de l'enfant, en particulier en ce qui concerne l'implication d'enfants dans les conflits armés.
Conformément à l'accord auquel le Conseil est parvenu lors de ses consultations préalables, j'invite les membres du Conseil à poursuivre l'examen de la question.
Décisions et recommandations


In [44]:
!python3 MT-Preparation/subwording/3-desubword.py target-general.model corpora/enfr/en-fr-general.fr-filtered.fr.subword.test

Done desubwording! Output: corpora/enfr/en-fr-general.fr-filtered.fr.subword.test.desubword


In [45]:
!head -n 5 corpora/enfr/en-fr-general.fr-filtered.fr.subword.test.desubword

Il ne faut pas oublier que 1 260 détenus marocains sont toujours en captivité dans les geôles du Polisario, et ce depuis plus de 25 ans, en violation flagrante du droit international humanitaire.
Faire parvenir au Comité des exemplaires du texte de la Convention relative aux droits de l'enfant dans toutes les langues officielles de l'État partie ainsi que dans ses autres langues ou dialectes, si elle a été traduite.
Nous réaffirmons que la législation en vigueur dans le Sultanat garantit la protection des droits de l'homme, y compris les droits de l'enfant, en particulier pour ce qui est de l'implication d'enfants dans les conflits armés.
Conformément à l'accord auquel le Conseil est parvenu lors de ses consultations préalables, j'invite à présent les membres du Conseil à poursuivre le débat sur la question dans le cadre de consultations.
Décisions et recommandations


In [22]:
# test bleu for baseline score
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
!pip3 install sacrebleu

--2024-01-24 02:43:49--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2024-01-24 02:43:49 (4.60 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [46]:
!python3 compute-bleu.py corpora/enfr/en-fr-general.fr-filtered.fr.subword.test.desubword general-fr.translated.desubword
# !python3 compute-bleu.py corpora/enfr/en-fr-general.en-filtered.en.subword.test.desubword general-fr.translated.desubword

Reference 1st sentence: Il ne faut pas oublier que 1 260 détenus marocains sont toujours en captivité dans les geôles du Polisario, et ce depuis plus de 25 ans, en violation flagrante du droit international humanitaire.
MTed 1st sentence: Il ne faut pas oublier que 1 260 prisonniers marocains sont toujours détenus dans des prisons du Front POLISARIO, où ils ont été maintenus pendant plus de 25 ans en violation flagrante du droit international humanitaire.
BLEU:  43.0993809551098


In [ ]:
tech = """# tech.yaml
# for technology corpus/model fine-tuning
share_vocab: true
src_vocab:
tgt_vocab:
data:
  # different corpus weighting for mixed fine-tuning approach
  tech_corpus:
    path_src:
    path_tgt:
    transforms:
    weight: 10
  gen_corpus:
    path_src:
    path_tgt:
    transforms:
    weight: 1

update_vocab: true
train_from: '' # the base model trained earlier
reset_optim: all

save_data:
save_model:
log_file:

keep_checkpoint: 50
save_checkpoint_steps: 100
average_decay: 0.0005
seed: 1234
report_every: 10

train_steps:
valid_steps:

# batching
bucket_size: 262144
num_workers: 0
"""

with open("tech.yaml", "w+") as tech_yaml:
  tech_yaml.write(tech)